# Lab 1: Build a Customer Support Agent with Claude Agent SDK

In this lab, you'll build a functional **Customer Support Agent prototype** using the **Claude Agent SDK** (`claude-agent-sdk`) with Amazon Bedrock.

## What you'll learn
- How to define custom tools using the Claude Agent SDK `@tool` decorator
- How to create an in-process MCP server with `create_sdk_mcp_server()`
- How to invoke the agent using `query()` with `ClaudeAgentOptions`
- How to configure the agent to use Bedrock via `CLAUDE_CODE_USE_BEDROCK=1`

## Architecture

![Architecture](images/architecture_lab1_claude.png)

## Key Differences from Strands Agents

| Strands Agents | Claude Agent SDK |
|---------------|------------------|
| `Agent(model, tools)(prompt)` | `query(prompt, options)` |
| `@strands.tools.tool` | `@claude_agent_sdk.tool` |
| `BedrockModel(model_id=...)` | `CLAUDE_CODE_USE_BEDROCK=1` env var |
| Tools passed directly to Agent | Tools wrapped in MCP server |

## Step 1: Install Dependencies

In [ ]:
%pip install -q -r requirements.txt

## Step 2: Configure Bedrock Backend

Claude Agent SDK uses the `CLAUDE_CODE_USE_BEDROCK=1` environment variable to route all model calls through Amazon Bedrock instead of the Anthropic API.

In [ ]:
import os
import json
import asyncio

# Configure Claude Agent SDK to use Bedrock
os.environ["CLAUDE_CODE_USE_BEDROCK"] = "1"

# Allow SDK to run (unset nested-session guard if present)
os.environ.pop("CLAUDECODE", None)

import boto3

region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity()["Account"]
print(f"Region: {region}")
print(f"Account: {account_id}")

## Step 3: Define Customer Support Tools

We define 4 tools using the Claude Agent SDK `@tool` decorator. Each tool has:
- A name and description for the model to understand when to use it
- An input schema defining expected parameters
- An async function that returns results in MCP-compatible format

In [ ]:
from agent.tools import (
    get_return_policy,
    get_product_info,
    web_search,
    get_technical_support,
    TOOLS,
)
from agent.prompts import SYSTEM_PROMPT, MODEL_ID

print(f"Model: {MODEL_ID}")
print(f"Tools: {[t.name for t in TOOLS]}")
print(f"\nSystem Prompt:\n{SYSTEM_PROMPT[:200]}...")

## Step 4: Create In-Process MCP Server

Claude Agent SDK uses MCP (Model Context Protocol) servers to expose tools. We wrap our tools in an in-process MCP server using `create_sdk_mcp_server()`.

In [ ]:
from agent.mcp_server import get_mcp_server

sdk_server = get_mcp_server()
print(f"MCP Server created: {sdk_server}")

## Step 5: Invoke the Agent

Now we use `query()` with `ClaudeAgentOptions` to invoke the agent. This is the core Claude Agent SDK pattern:

```python
options = ClaudeAgentOptions(
    system_prompt=SYSTEM_PROMPT,
    mcp_servers={"customer-support": sdk_server},
    allowed_tools=["mcp__customer-support__*"],
    max_turns=10,
)

async for msg in query(prompt=prompt_stream(user_input), options=options):
    if hasattr(msg, 'result'):
        print(msg.result)
```

**Note:** We use `prompt_stream()` (an async generator) instead of passing a raw string. This ensures the SDK keeps stdin open for bidirectional MCP server communication. The `prompt_stream` helper is defined in `agent/__init__.py`.

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions
from agent import prompt_stream


async def invoke_agent(user_input: str) -> str:
    """Invoke the customer support agent with a user query."""
    options = ClaudeAgentOptions(
        system_prompt=SYSTEM_PROMPT,
        mcp_servers={"customer-support": sdk_server},
        allowed_tools=["mcp__customer-support__*"],
        permission_mode="bypassPermissions",
        max_turns=10,
    )

    response_text = ""
    async for msg in query(prompt=prompt_stream(user_input), options=options):
        if hasattr(msg, "result"):
            response_text = msg.result

    return response_text

## Step 6: Test the Agent

Let's test with several customer support scenarios.

In [ ]:
# Test 1: Return policy inquiry
response = await invoke_agent("What is your return policy for laptops?")
print("=" * 60)
print("Query: What is your return policy for laptops?")
print("=" * 60)
print(response)

In [ ]:
# Test 2: Product information
response = await invoke_agent("Tell me about your smartphones. What are the key features?")
print("=" * 60)
print("Query: Tell me about your smartphones. What are the key features?")
print("=" * 60)
print(response)

In [ ]:
# Test 3: Web search
response = await invoke_agent("What are the latest Wi-Fi 7 routers available in 2025?")
print("=" * 60)
print("Query: What are the latest Wi-Fi 7 routers available in 2025?")
print("=" * 60)
print(response)

In [ ]:
# Test 4: Technical support (requires Knowledge Base from prerequisites)
try:
    response = await invoke_agent("My laptop won't turn on after a software update. What should I do?")
    print("=" * 60)
    print("Query: My laptop won't turn on after a software update. What should I do?")
    print("=" * 60)
    print(response)
except Exception as e:
    print(f"Note: Technical support requires Knowledge Base setup from prerequisites.")
    print(f"Error: {e}")

## Summary

In this lab, you built a customer support agent using Claude Agent SDK with:

1. **4 custom tools** defined with `@tool` decorator
2. **In-process MCP server** via `create_sdk_mcp_server()`
3. **Agent invocation** via `query()` with `ClaudeAgentOptions`
4. **Bedrock backend** via `CLAUDE_CODE_USE_BEDROCK=1`

### Limitations of this prototype
- No memory - the agent doesn't remember previous conversations
- Single user - no multi-tenant support
- Local only - not deployed for production use

In **Lab 2**, we'll add AgentCore Memory to give the agent persistent memory across conversations.